In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [2]:
df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_All_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zchodaniecky\\OneDrive - Franklin Templeton\\Documents\\Python\\NHL_data\\NHL_Data_All_Transformed.csv'

In [ ]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [ ]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [ ]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [ ]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

In [ ]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [ ]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [ ]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [ ]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [ ]:
# Set parameters
hidden_layers = 0
neurons = 35
learning_rate = 0.00039447935272202665
batch_size = 5
n_epochs = 80

In [ ]:
def build_model(n_hidden=hidden_layers, n_neurons=neurons, learning_rate=learning_rate, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=hidden_layers, n_neurons=neurons, learning_rate=learning_rate)


In [ ]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, epochs=n_epochs, batch_size=batch_size, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')


In [ ]:
# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("Best_NHL_Model.keras", save_best_only = True)
model_history = keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
                              callbacks=[early_stopping_cb]).history_


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# print(model_history.keys())

# pd.DataFrame(model_history['loss']).plot(figsize=(8,5))
# plt.grid(True)
# plt.gca().set_ylim(0.58,0.61) # set the vertical range to [0-1]
# plt.show()

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


In [ ]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

In [ ]:
df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [ ]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T